In [ ]:
import pyspark.sql.functions as f

In PySpark DataFrames - Part 2 module you've learned how to join dataframes, perform aggregations and create pivot tables to answer business questions

In this notebook there are some more of those questions that you can answer using PySpark DataFrames methods and SQL functions.

Some of them will require you to look into the documentation to find the right function to use, which I think is the best way to learn how to use PySpark.

So, let's get started!

First, get the **preprocessed orders and products DataFrames** like you did in the notebook.

In [ ]:
df_orders = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ",")
    .load("/FileStore/lp-big-data/orders-data/orders-preprocessed.csv")
)

df_products = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .option("sep", ",")
    .load("/FileStore/lp-big-data/orders-data/products-preprocessed.csv")
)

Join the tables on the product_id column and store the result in a new dataframe called orders_products.

For the exercises, ignore all orders that don't have a matching product in the products dataframe.

Then, show the first 5 rows of the orders_products dataframe.

In [ ]:
df_orders_products = (
    df_orders.join(
        df_products,
        on=['product_id'],
        how='inner'
    )
)

df_orders_products.show(5)

1. Do fast deliveries have more revenue than the other ones? And are they more profitable?

In [ ]:
(
    df_orders_products
    # .withColumn('total_price', f.col('cost_per_unit') * f.col('amount'))
    .groupBy('delivery_speed')
    .agg({
        'revenue': 'avg',
        'profit': 'avg',
    })
).display()

2. What is the average lead time of each product considering only the 100 most profitable products?

***Hint:*** Break down the problem into smaller steps.

First, calculate the average profit and lead time for each product
Then select only the 100 most profitable products and select the average lead time for them.

In [ ]:
(
    df_orders_products
    .groupBy('product_id')
    .agg(
        f.avg('profit').alias('avg_profit'),
        f.avg('lead_time').alias('avg_lead_time'),
    )
    .orderBy(f.desc('avg_profit'))
    .limit(100)
    .select(
        'product_id',
        'avg_lead_time'
    )
).display()

3. What is the variance of total revenue generated per month in each country?

In [ ]:
(
    df_orders_products
    .groupBy(['order_month', 'supplier_country'])
    .agg(f.variance('revenue').alias('revenue_variance'))
).display()

4. How many orders of each product category were placed, considering only orders with a number of days to delivery slower than the median?

***Hint:*** Start by calculating the median delivery speed separately. Look into the [documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html) to find the right function to use.

In [ ]:
median_delivery_speed = df_orders_products.stat.approxQuantile('days_to_delivery', [0.5], 0.01)[0]

(
    df_orders_products
    .filter(f.col('days_to_delivery') > median_delivery_speed)
    .groupBy('product_category')
    .count()
).display()

5. How many orders were placed on each day of the week, considering only orders placed in the first half of each year?

In [ ]:
(
    df_orders_products
    .filter(f.col('order_month') <= 6)
    .groupBy('order_day_of_week')
    .count()
).display()

6. How many unique products were announced in each supplier continent before 15-05-2014?

In [ ]:
(
    df_orders_products
    .filter(
        f.col('announcement_date') < f.to_date(f.lit('2014-05-15'))
    )
    .groupBy('supplier_continent')
    .agg(
        f.countDistinct('product_id').alias('number_of_products')
    )
).display()

7. How many orders in each supplier country had a delivery delay greater than the global average delivery delay?

In [ ]:
# Convert delivery date to unix timestamp to apply the average function
avg_delivery_date = (
    df_orders_products
    .select(f.avg(f.col('days_to_delivery')))
    .first()
)[0]

(
    df_orders_products
    .filter(f.col('days_to_delivery') > avg_delivery_date)
    .groupBy('supplier_country')
    .count()
).display()